# Result-Dependent Loops

To iterate over the result from an electron, put the iteration (loop) logic inside another electron.

## Context

Often the output of one task is a collection that you want to iterate over using another task. In Covalent terms, this means you want to use an `electron` to produce an `iterable`, then process the iterable with another electron. In these cases, perform the iteration inside an electron. 

When a lattice is dispatched, the Covalent server executes the lattice in order to build the transport graph. The transport graph is then analyzed to parallelize the execution of electrons on their assigned executors.

If the server encounters a loop over the output of an electron, it cannot infer the structure on which the loop depends (the size of the iterable  and must wait uis prevented from building the transport graph.


Note: This pattern applies only when the iterator is produced by an electron. Iterating on fixed values in a lattice as described [here](https://covalent.readthedocs.io/en/latest/how_to/coding/looping.html) does not require electron execution to evaluate the iterator and build the graph.

## Best Practice

Compute dynamically generated iterators inside an electron. Electrons execution is deferred during the graph build phase, so they cannot be used to build the transport graph and analyze the execution for parallelization.

**Exception**: Sublattices are evaluated during lattice execution and their graph structure is "plugged in" to the larger transport graph. If a sublattice produces a static iterator (does not produce it by executing an electron internally), then it is safe to loop over.

## Example

Contrast the two examples below.

### Not Recommended

This example demonstrates a questionable approach: looping over a computed iterator in the lattice but not within an electron.

In [11]:
import covalent as ct
import random

# Technique 1: inefficient

@ct.electron
def task():
    return random.sample(range(10, 30), 5)

@ct.electron
def task_2(x):
    return x ** 2

@ct.lattice
def workflow_1():
    random_list = task()

    res = []
    for rn in random_list:
        res.append(task_2(rn))
    
    return rn

id = ct.dispatch(workflow_1)()
res = ct.get_result(id, wait=True)

# Improved

The iterator is passed to the second electron, which loops over it internally and returns the results in a list. In this case the loop is executed entirely at runtime.

In [12]:
import covalent as ct
import random

# Technique 2: iterator is contained in an electron

@ct.electron
def task_1():
    return random.sample(range(10, 30), 5)

# Method (2):
@ct.electron
def task_2_new(x_list):

    squares = []
    for x in x_list:
        squares.append(x ** 2)
    
    return squares

@ct.lattice
def workflow_2():
    random_list = task_1()
    return task_2_new(random_list)

id = ct.dispatch(workflow_2)()
res = ct.get_result(id, wait=True)

## Keywords

Typically glossary terms that might be useful in looking up the technique.

## Benefits

Typical benefits that might appear here:
- Performance
- Simplicity
- Maintainability
- Resource (CPU, memory) conservation
- Efficiency
- Workaround (to a problem with Python or with Covalent)

## See Also